# GNN for NLP
* STC-SKB-GRAPH-Ohsumed
---

In [1]:
import tqdm
import numpy as np
import json
import spacy
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Short Text Classification (STC)

In [2]:
STC_Benchmark_path = "../benchmark"
Ohsumed_training: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_training.npy", allow_pickle=True).tolist()
Ohsumed_test: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_test.npy", allow_pickle=True).tolist()
Ohsumed_category_description: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_category_description.npy", allow_pickle=True).tolist()
Ohsumed_ohsumed_all: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_ohsumed_all.npy", allow_pickle=True).tolist()

In [3]:
Ohsumed_ohsumed_all_tagme: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_ohsumed_all_tagme.npy", allow_pickle=True).tolist()
Ohsumed_training_tagme: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_training_tagme.npy", allow_pickle=True).tolist()
Ohsumed_test_tagme: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_test_tagme.npy", allow_pickle=True).tolist()
Ohsumed_category_description_tagme: dict = np.load(STC_Benchmark_path+"/Ohsumed/Ohsumed_category_description_tagme.npy", allow_pickle=True).tolist()

In [4]:
def tagmeProcess(tagme_json):
    tagme_dict = {}
    for items in tqdm.tqdm(tagme_json):
        if not items:
            continue 
        i, entityList = items.split("\t")
        if entityList == "null" or len(entityList) == 0:
            continue
        entityList = json.loads(entityList)
        entities_spot = [d['spot'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
        entities_title = [d['title'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
        entities = (entities_spot,entities_title)
        if i not in tagme_dict.keys():
            tagme_dict[i] = []
        tagme_dict[i] = entities
    return tagme_dict

In [5]:
Ohsumed_ohsumed_all_tagme = tagmeProcess(Ohsumed_ohsumed_all_tagme)
Ohsumed_training_tagme = tagmeProcess(Ohsumed_training_tagme)
Ohsumed_test_tagme = tagmeProcess(Ohsumed_test_tagme)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4043/4043 [00:00<00:00, 73314.19it/s]


In [6]:
print(sum([len(t) for (_,t) in Ohsumed_ohsumed_all_tagme.values()])/len(Ohsumed_ohsumed_all))
print(sum([len(t) for (_,t) in Ohsumed_training_tagme.values()])/len(Ohsumed_training))
print(sum([len(t) for (_,t) in Ohsumed_test_tagme.values()])/len(Ohsumed_test))

4.9846380931554775
4.867441167709265
4.9121939154093495


In [7]:
print(sum([len(c.split(" ")) for c in Ohsumed_ohsumed_all.keys()])/len(Ohsumed_ohsumed_all))
print(sum([len(c.split(" ")) for c in Ohsumed_training.keys()])/len(Ohsumed_training))
print(sum([len(c.split(" ")) for c in Ohsumed_test.keys()])/len(Ohsumed_test))

12.106986660835338
11.820971105153411
12.016077170418006


In [8]:
#list(Ohsumed_training.keys())[5]

In [9]:
#list(Ohsumed_training_tagme.values())[5]

In [10]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemmatization(sentence):
    tokens = word_tokenize(sentence)
    tagged_sent = pos_tag(tokens)
    wnl = WordNetLemmatizer()
    lemmas_sent = []
    for tag in tagged_sent:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmas_sent.append((wnl.lemmatize(tag[0], pos = wordnet_pos),wordnet_pos))
    return lemmas_sent

## Analysis for OOV on dataset

In [22]:
def wordFreq(docs):
    word_freq = {}
    for doc in docs:
        for word in doc.split():
            if word in word_freq:
                word_freq[word] += 1
            else:
                word_freq[word] = 1
    return word_freq
def evaluationFrequncy(docs,save_name,limit_num):
    '''evaluation frequncy for document lexions'''
    word_freq = wordFreq(docs)
    print("=======analysis start=======")
    print("#all word size: ", len(word_freq))
    limit_word_freq_len = len([v for k,v in word_freq.items() if v < limit_num])
    word_freq_sorted = sorted(word_freq.items(), key = lambda kv:(kv[1], kv[0]))
    print("#frequncy < "+ str(limit_num) +": ", limit_word_freq_len)
    print("#frequncy mean: ", np.mean(list(word_freq.values())))
    print("#frequncy standard deviation: ", np.std(list(word_freq.values())))
    print("#frequncy std/mean: ", np.std(list(word_freq.values()))/np.mean(list(word_freq.values())))
    #axes = sns.scatterplot(data=list(word_freq.values())).set_title(save_name)
    #axes.figure.set_size_inches(18,4)
    #fig = axes.get_figure()
    #fig.savefig("../data/images/"+save_name+".png", dpi = 400)
    return word_freq, word_freq_sorted

In [17]:
Ohsumed = dict(Ohsumed_test,**Ohsumed_training)

In [34]:
Ohsumed_list = [k for k,v in Ohsumed_test.items()] + [k for k,v in Ohsumed_training.items()]

In [36]:
np.save("Ohsumed_stc_list",Ohsumed_list)

In [28]:
raw_word_freq, raw_word_freq_sorted = evaluationFrequncy(Ohsumed_list,"raw_ohsumed",limit_num=5)

=======analysis start=======
#all word size:  18346
#frequncy < 5:  15710
#frequncy mean:  4.809331734438024
#frequncy standard deviation:  65.32569453623601
#frequncy std/mean:  13.58311261177108


In [ ]:
doc_content_tuple = (raw_word_freq,raw_word_freq_sorted,Ohsumed_list)
np.save("../data/corpus/Ohsumed_stc_tuple", doc_content_tuple)

In [ ]:
Ohsumed_ohsumed_all_tagme

## Get the DictSKB and own SKB-DA

In [14]:
dictskb_dict = np.load("../sememe_dataset/DictSKB_dict.npy", allow_pickle=True).tolist()
dictskb_sememes = np.load("../sememe_dataset/DictSKB_sememes.npy", allow_pickle=True).tolist()

In [15]:
skb_ad_dict = np.load("../sememe_dataset/skb_ad_dict.npy", allow_pickle=True).tolist()

In [70]:
list(Ohsumed_training.items())[15]

('The national immunization program of The Netherlands.', ['C01'])

In [71]:
dictskb_dict["immunization"]

[('verb', {'protect'})]

In [72]:
skb_ad_dict["immunization"]

[('NN', {'agent', 'immune', 'infectious', 'process'}), ('n', {'immune'})]